In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install youtube-dl

# PILIH YOUTUBE URL

In [90]:
url_yt = 'https://www.youtube.com/watch?time_continue=10&v=IfHadadE-TtMJJw'

In [92]:
import subprocess
param = "python -m youtube_dl --restrict-filenames --ignore-errors -x --audio-format mp3 -o output.mp3 "

if (subprocess.call(param+url_yt) == 0):
    print("Download youtube ke mp3 - SUCCESS")
else:
    print("Download youtube ke mp3 - GAGAL")

Download youtube ke mp3 - GAGAL


## KONVERSI KE WAV
Harus ada FFMPEG di PATH
- step 1 - download file dr source dalam bentuk mp3
- step 2 - convert WAV dengan audacity
- step 3 - hapus directory input.

In [96]:
input_file = 'output.mp3'

In [97]:
!echo Y | del .\input\*

E:\PYTHON\transcribe\input\*, Are you sure (Y/N)? Y 


### step 4 - Potong menjadi 1 menit tiap chunk. - kalau return 0 berarti berhasil  -acodec pcm_u8 -ac 1 

In [98]:
import subprocess
convert = f"ffmpeg.exe -i {input_file} -f segment -segment_time 30 -c copy -acodec pcm_s16le -ar 44100 -ac 1 input/trans%09d.wav"

if (subprocess.call(convert) == 0):
    print("KONVERSI KE WAV - SUCCESS")
else:
    print("KONVERSI KE WAV - ERROR")

KONVERSI KE WAV - SUCCESS


In [102]:
import os
import speech_recognition as sr
from tqdm import tqdm
from multiprocessing.dummy import Pool
import glob
pool = Pool(8) # Number of concurrent threads

In [103]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "api-key.json"

## 1 . CARA PERTAMA - MultiThread multiple files

In [104]:
files = sorted(glob.glob('./input/*.wav'))
language='id-ID'

In [105]:
files

['./input\\trans000000000.wav',
 './input\\trans000000001.wav',
 './input\\trans000000002.wav',
 './input\\trans000000003.wav',
 './input\\trans000000004.wav',
 './input\\trans000000005.wav',
 './input\\trans000000006.wav',
 './input\\trans000000007.wav',
 './input\\trans000000008.wav',
 './input\\trans000000009.wav',
 './input\\trans000000010.wav']

In [ ]:
r = sr.Recognizer()

def transcribe(data):
    idx, file = data
    name = file
    print(name + " started\n")
    # Load audio file
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcribe audio file
    try:
        text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS,language=language)
    except:
        return {
            "idx": idx,
            "text": ""
        }
    print(name + " done")
    return {
        "idx": idx,
        "text": text
    }

all_text = pool.map(transcribe, enumerate(files))
pool.close()
pool.join()

transcript = ""
for t in sorted(all_text, key=lambda x: x['idx']):
    total_seconds = t['idx'] * 30
    # Cool shortcut from:
    # https://stackoverflow.com/questions/775049/python-time-seconds-to-hms
    # to get hours, minutes and seconds
    m, s = divmod(total_seconds, 60)
    h, m = divmod(m, 60)

    # Format time as h:m:s - 30 seconds of text
    transcript = transcript + "{:0>2d}:{:0>2d}:{:0>2d}\n {}\n\n".format(h, m, s, t['text'])

print(transcript)

with open(long_filename+".txt", "w") as f:
    f.write(transcript)

## 2. CARA KE DUA - Single file WAV Long Recognizer

In [108]:
filename = './input/trans000000000.wav'

In [109]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
import io


def sample_long_running_recognize(local_file_path):
    """
    Transcribe a long audio file using asynchronous speech recognition

    Args:
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """

    client = speech_v1.SpeechClient()

    # local_file_path = './input/trans000000000.wav'

    # The language of the supplied audio
    language_code = "id-ID"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
    }
    with io.open(local_file_path, "rb") as f:
        content = f.read()
    audio = {"content": content}

    operation = client.long_running_recognize(config, audio)

    print(u"Waiting for operation to complete...")
    response = operation.result()

    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        

sample_long_running_recognize(filename)

Waiting for operation to complete...
Transcript: hari ini kami sudah melakukan pemeriksaan spesimen sebanyak 11361
Transcript:  singkat total keseluruhan adalah rp311.000 6 spesimen
Transcript:  dari pemeriksaan ini kita dapatkan kasus konfirmasi positif naik 557 sehingga menjadi 25773
